![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/open-source-nlp/14.0.GPT2_Transformer_In_SparkNLP.ipynb)

# GPT2Transformer: OpenAI Text-To-Text Transformer

![image.png](https://miro.medium.com/max/1400/1*UZTblFMn8oWfmqBMQhUOdA.jpeg)

[GPT-2](https://openai.com/blog/better-language-models/)  displays a broad set of capabilities, including the ability to generate conditional synthetic text samples of unprecedented quality, where the model is primed with an input and it generates a lengthy continuation.

Pretrained models can be loaded with `pretrained()` of the companion object:

**GPU MODE** is highly reccomended, since this is a very large model.

## Colab Setup

In [ ]:
! pip install -q pyspark==3.3.0 spark-nlp==5.0.0

In [ ]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

**Make sure to Enable GPU Mode and High RAM**

In [ ]:
# Comment out this line  and uncomment the next one to enable GPU mode and High RAM
#
spark = sparknlp.start()

# spark = sparknlp.start(gpu=True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 5.0.0
Apache Spark version: 3.3.0


**GPT2 Models In Spark NLP**

*   `gpt2`
*   `gpt2_medium`
*   `gpt2_distilled`
*   `gpt2_large`

The default model is `"gpt2"`, if no name is provided. For available pretrained models please see the [Spark NLP Models Hub](https://nlp.johnsnowlabs.com/models?q=gpt2)



## GPT2 Pipeline

Now, let's create a Spark NLP Pipeline with `gpt2_medium` model and check the results.

In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

gpt2 = GPT2Transformer.pretrained("gpt2_medium") \
    .setInputCols(["documents"]) \
    .setMaxOutputLength(50) \
    .setMinOutputLength(25) \
    .setOutputCol("generation")

data = spark.createDataFrame([["My name is Leonardo."]]).toDF("text")

pipeline = Pipeline().setStages([documentAssembler,gpt2]).fit(data)

result = pipeline.transform(data)
result.select("generation.result").show(truncate=False)

gpt2_medium download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                           |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[ My name is Leonardo. I'm a student at the University of California, Berkeley. I've been studying computer science for the past two years. I have a PhD in computer scienc

We can display the documentation of all params with their optionally default values and user-supplied values by `explainParams()` function

In [ ]:
print(gpt2.explainParams())

batchSize: Size of every batch (default: 4)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: False)
engine: Deep Learning engine used for this model (default: tensorflow)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxOutputLength: Maximum length of output text (default: 20, current: 50)
minOutputLength: Minimum length of the sequence to be generated (default: 0, current: 25)
noRepeatNgramSize: If set to int > 0, all ngrams of that size can only occur once (default: 0, current: 3)
outputCol: output annotation column. can be left default. (current: generation)
repetitionPenalty: The parameter for

Let's use model with more sentences and set `.setDoSample()` parameter as True, this parameter is used for whether or not to use sampling; use greedy decoding otherwise, by default False. <br/>
Also, we use `.setTopK()` parameter for the number of highest probability vocabulary tokens to keep for top-k-filtering, by default 50.

In [ ]:
sample_texts= [[1, "Mey name is  Leonardo"],
               [2, "My name is Leonardo and I come from Rome."],
               [3, "My name is"],
               [4, "What is the difference between diesel and petrol?"]]

sample_df= spark.createDataFrame(sample_texts).toDF("id", "text")

In [ ]:
gpt2.setMaxOutputLength(50).setMinOutputLength(25).setDoSample(True).setTopK(20)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
result = pipeline.transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                                                  |
+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[ Mey name is  Leonardo da Vigo and I have met him before in Madrid, and that is why I did this project. This was to be a project that I could show how I can be involved with the local and global movement that is currently]         |
|2  |[ My name is Leonardo and I come from R

### Changing the Transformer's task

Now, we change the task of Transformer. We can verify some informations to GPT-2 by setting `.setTask()` parameter as **"Is it true that"**. <br/>
We give a text to the model by setting `setTask("Is it true that")` and model adds the "Is it true that" expression at the beginning of the sentence and generates sentences.

In [ ]:
gpt2.setTask("Is it true that").setMaxOutputLength(50)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
sample_text= [[1, "Donald Trump is rich?"],
              [2, "Pink Floyd is rock band?"]]

sample_df= spark.createDataFrame(sample_text).toDF("id", "text")

result = pipeline.transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                                           |
+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[ Is it true that Donald Trump is rich? I'll take you at your word for it.\n\nThe Trump Organization is a real estate conglomerate and the CEO is Donald Trump. His name isn't actually the owner's name as it would be if Trump]|
|2  |[ Is it true that Pink Floyd is rock band? Are they a rock band or 

# Some Helper Methods for generating and printing generation results

In [ ]:
from typing import List
def prompts_to_spark_df(prompts,spark=spark):
  text = [[i, prompt] for i,prompt in enumerate(prompts)]
  return spark.createDataFrame(text).toDF("id", "text")

def generate_with_gpt2(gpt_pipe, prompts,task='generate', print=True):
  if isinstance(prompts,str):
    df = prompts_to_spark_df([prompts])
  elif isinstance(prompts,List):
    df = prompts_to_spark_df(prompts)
  else :
    raise ValueError(f"Invalid Type = {type(prompts)} please pass a str or list of str for prompts parameter ")
  df = pipeline.transform(df)
  df = df.select("id",'text', "generation.result").toPandas()

  if print:
    print_generation_results(df)
  return df

def print_generation_results(df):
  for idx, row in df.iterrows():
    print(f'Example {idx}: {200*"_"}')
    print(row.result[0])
    print('\n')


# Explore GPT Parameters Play with Paramns

### Sampling Methods


Sampling means we **randomly** draw from a distribution of words.
The probability distribution is conditioned on all previous tokens in a text to generate the next token.

By default the distribution contains all words in the vocabulary of GPT2, where many candidates are incorrect to generate.

There are two methods of reshaping and drawing from those distributions :

1. **Top-K Sampling** Take the k most likely words from the original distribution. Redistribute probability mass among those k words and draw according to the new probabilities.

2. **Top-P Nucleus sampling**  Take smallest possible set of N words, which  together have a probability of p. Redistribute probability mass among those N words and draw according to the new probabilities.



Additionally, both methods can be tweaked ith the following parameters :

- **temperature** : Parameter of the softmax function which affect the distrubtion computed by the model. The closer we are to 0, the more deterministic the probability will become, distribution tails will become slimmer and outlier word probabilites are more close to 0. Temperature values closer values to 1 make tails of probability fatter which makes outliers more probable and generic results less probable.


These parameters are shared by all method :
- **ignoreTokenIds**: A list of token ids which are ignored in the decoder's output (default: [])
- **noRepeatNgramSize**: If set to int > 0, all ngrams of that size can only occur once
- **repetitionPenalty**: The parameter for repetition penalty. 1.0 means no penalty.  https://arxiv.org/pdf/1909.05858.pdf>
- **task**:  Transformer's task, e.g. 'is it true that'> (default: , current: generate)

In [ ]:
print(gpt2.explainParams())

batchSize: Size of every batch (default: 4)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: True)
engine: Deep Learning engine used for this model (default: tensorflow)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxOutputLength: Maximum length of output text (default: 20, current: 50)
minOutputLength: Minimum length of the sequence to be generated (default: 0, current: 25)
noRepeatNgramSize: If set to int > 0, all ngrams of that size can only occur once (default: 0, current: 3)
outputCol: output annotation column. can be left default. (current: generation)
repetitionPenalty: The parameter for 

### Play with temperature
Set Temperature higher to make GPT more random/creative and text less coherent
Temperature > 0  and Temperature <=1
You must set `gpt2.setDoSample(True)` to have non-deterministic results

In [ ]:
text = """Hello my name is GPT2, I love to """
data = [text, text,text,text,text ]
gpt2.setMaxOutputLength(200)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !"

This is the official Japanese announcement: http://www.facebook.com/Hello-my-NAME-is-GPT2-I-love-to-I/1645692949682637


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to irl do it? Is it possible to be a goth by going naked and being sadistic with yourself? Is there a G.P.T. 2 version of me that you can use? Are there any people who you admire that aren't goth?

The answer is yes. And this is what I wanted to address. The G.S.T.'s are different, but not by a huge amount. They're also not completely dissimilar from o

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.75)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to _____
The first step is to select the folder you want to store the files in. Open up a folder and copy-paste the following to the file explorer and paste it in the folder:
cd /my/GPT2/GUID
Now that you have the files there, it's time to extract them to a directory. Go back to the original folder on your computer, and make sure the GPT files are in it. On your Linux computer, open a terminal and cd into the directory you just created. In the terminal, type the following in the terminal:
sudo mkdir /tmp/GPS2.log
This will create a copy of the GZIP file that you want the files to be copied to, in this case, /tmp. In this case I'm going to use the file location of /tmp and put the GPS files into it. It's not


Example 1: ___________

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.5)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to ____, I like to ______, I'm a ______, and I'm ______?

The answer is yes, and the answer to the question is yes.

Yes, the answer is "yes."

In fact, it is not just that the answer was "yes" that made this answer so interesting.
.
 of the two options, the one that is more likely to be considered "yes," is the one which is more often associated with the "pornographic" meaning of the term.
 and the one associated with "purity."
.


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!!

The first time I heard the word GPT w

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.25)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline,data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!!

Hello my name, I am GPT1, I'm a girl, I like to !!!!
 .
 1. I love !!! 2; I am !!! 3; I love you !!! 4; I want !!! 5 (I want to !!); I want to be !!! 6; I like !!! 7; I'm !!! 8; I don''t !!! 9; I think !!! 10; I have !!! 11; I will !!! 12; I was !!! 13; I hope *********** 14; I feel !!! 15; I know !!! 16; I can !!! 17; I need !!! 18; I hate !!! 19; I wish !!! 20; I would !!! 21; I'd like !!!! 22; I really !!! 23; I'll !!! 24; I've !!! 25;


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!!

I'm GPT1, I'm GTA1, a

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!! But is it true? Can I install some packages of Debian 8 by a community software to work my Ubuntu/Mac systems from here? It is, this site also offer, in general, several of its projects under license GNU GPLv3. Are you in trouble as GPT1 in all Linux Operating Systems? In a previous FAQ on linux.die, is it right to change default font color using dconf-editor / dconf? Please ask again to us (Dmitry) I see that dconf is running inside Windows or even using Linux environment at http / www .. ? The difference between GPT and GPT-DEFAULT_COMPONENT or GPAGES is about 4K x4K / 100K pages to make one Debian Debian/Ubare or Mac OS X Server desktop display all Ubuntu GPT file as gpt / etc / pkgs / x86_64


Example 1: ________________

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.0001)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to ________________, HiGTPiRzCq4Nc7H3R7c1jdPtPX9P5p4x
It does however work just okay so its definitely more like how an eee can feel with gppa . Not great yet for someone to install from usb though if only one guy or so gets hold in him!So that might ormay explain, just so no surprises:My eemup does appear better however...it appears slightly weaker dueto what a bunch nuke said that we already already found here as stated a previous week..If they will keep talking then more testing are surely being ordered soon enoughI've made more sure, thanks very kindly people are interested :),Hope everything go very right this new development cycle in linux . This project also deserves our applause if i'll continue its working !It just got ver

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


# GPT2 Use Cases

These are some examples to give you an idea of what GPT2 is capable of

## GPT for Programming Code

### GPT and Scala

In [ ]:
# Hello world in Scala
scala_code = """
object Hello {
    def main(args: Array[String]) = {
        println("Hello, world")
    }
}
"""
gpt2.setMaxOutputLength(1000)

gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, scala_code ,task='generation', print=True)



Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
object Hello {
    def main(args: Array[String]) = {
        println("Hello, world")
    }
}
If you're familiar with the language, it'll look like this:
Hello,world
This isn't particularly surprising when you understand that the first line is a call to the Hello type and the second is a method invocation. This is a way to define functions as parameters to other functions. The type of the argument is not known yet and therefore can be inferred as Object .
Here is a simple example that can be used to build an app to demonstrate what you're talking about.
1 2 3 4 5 6 7 8 class Program { def __init__(self)(args: Int): Unit = { self.log() } # ... def run(self) : Unit = super(*self).run(args) } import System.Reflection.Generic val app : App = new App() app.run() // 

,id,text,result
0,0,\nobject Hello {\n def main(args: Array[Str...,[ Is it true that \nobject Hello {\n def ma...


### GPT and Python

In [ ]:
# adding 3 numbers in python
py_code = """
def my_func(a,b,c):
  return a+b+c
"""
gpt2.setMaxOutputLength(1000)

gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, py_code ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
def my_func(a,b,c):
  return a+b+c
If you're not using a dictionary, you should also use the dictionary's __get__ method, but you'll have to modify your code a bit.
The dictionary example uses the __get_class__ methods from the __dict__ class.
__get_func()
def __get() :
__dict__. __getattr__ ( self .my_class_name, _ " __get " , False )
__init__()
__set_property__( " my_class " , self . my_value )
def get_func_class ( a = None ):
if a in _my_dict():
a = my_my_value. __default__
elif not a in __get:
raise NotImplementedError ( ' class must be specified. %s ( %r ) ' % (a, _my__class))
_get_property('my_property', a)
__doc__
def _method
__def __init__(self):
__self.___docname = "get_myclass"
__del__(__get__)
def return_func ( self ):
raise ValueError ( " Return no

,id,text,result
0,0,"\ndef my_func(a,b,c):\n return a+b+c\n","[ Is it true that \ndef my_func(a,b,c):\n ret..."


In [ ]:
# breadth-first search
bfs_python_code = """
def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0)
    print (s, end = " ")

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)
"""


gpt2.setMaxOutputLength(1000)
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, bfs_python_code ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0)
    print (s, end = " ")

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)





,id,text,result
0,0,"\ndef bfs(visited, graph, node):\n visited.ap...","[ Is it true that \ndef bfs(visited, graph, no..."


## Generate Product Recommendations

We can start a base list of top 10 anything and GPT2 will complete it for us!      
This is very useful to reccomendation and search engines.

### Top 10 Movie List

In [ ]:
movie_list = """My top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4."""

gpt2.setTemperature(1)
gpt2.setMaxOutputLength(200)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4. Star Wars 1
5. Star Trek 1
6. The Avengers 1
7. Back to the Future 1
8. The Dark Knight Rises 1
9. The Chronicles of Narnia 1
10. Star Blazers
Now it's time to see a list of the top 10 movies of 2001. So which do you think is the best? Let's look into it.




,id,text,result
0,0,My top 10 movie list :\n1. The Matrix 1\n2. Th...,[ Is it true that My top 10 movie list :\n1. T...


How **not** to generate text with GPT2

Since every new word is conditioned on every previous **character**, things like **white spaces** and **new lines** can skew the sampling distribution into an unintended direction

In [ ]:
# NOTE! We have malformed the input text and added a newline.
# This will Confuse GPT2, it might think the list is over and probability distribution changes accordingly
movie_list = """My Top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4.

"""
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4.


How does a movie like this compare to others. Is it better?

Yes.. this movie looks amazing

No.. this is not even close

Does this movie have any special effects

YES, the gun is amazing!

Not so much but the camera is amazing too

Would I watch it again? Yes.. the ending of the movie is very interesting.

How would I react if I was in a movie theater with you and said "This is so cool! I want to see it again!"

The fact that they are all about sex and violence is amazing



Why doesn't The Matrix have any romance in it?
:)

I have been to more movies with a romance plot but no climax.. The only other movie is Good Will Hunting which has two lovers and




,id,text,result
0,0,My Top 10 movie list :\n1. The Matrix 1\n2. Th...,[ Is it true that My Top 10 movie list :\n1. T...


In [ ]:
movie_list = """My Top 10 worst movie list :
1. The Matrix 4
2. Jack and Jill
3. Super Mario Bros. (1993)
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 worst movie list :
1. The Matrix 4
2. Jack and Jill
3. Super Mario Bros. (1993)
4.  The Matrix 2




,id,text,result
0,0,My Top 10 worst movie list :\n1. The Matrix 4\...,[ Is it true that My Top 10 worst movie list :...


### Top 10 Game List

In [ ]:
game_list = """My Top 10 video games of all time list :
1. Half-Life 2
2. Super Smash Bros Melee
3. Portal
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, game_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 video games of all time list :
1. Half-Life 2
2. Super Smash Bros Melee
3. Portal
4.  Doom (PS2)
5.   Super Metroid
6.    Doom3 (DS)
7.     Final Fantasy VII
8. 『Tekken'』
9. ��� Grand Theft Auto IV
10. Â ��� Final Fantasy XII
11. �� �� Final Fantasy IV
12. _________ Final Fantasy II
13. �  Final Fantasy II


This is the top 25 game list from the first video games magazine, which was published in Japan in the mid-1990s. The Top 25 is based on a list based on how many players played each game that year. This list was made up of those games that were popular for the most part in Japan, but some other games were not widely recognized in




,id,text,result
0,0,My Top 10 video games of all time list :\n1. H...,[ Is it true that My Top 10 video games of all...


### Top 10 Book List

In [ ]:
game_list = """My Top 10 books of all time list :
1. 1964
2. Moby Dick
3. robinson crusoe
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, game_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 books of all time list :
1. 1964
2. Moby Dick
3. robinson crusoe
4.  The Great Pyramids of Giza
5.  The Lord of the Rings
6.  Harry Potter
7.   The Adventures of  Tom Sawyer
8. The Hitchhikers Guide to the Galaxy
9. The Lord Of The Rings: Return of the King
10. Star Wars
The book list was made in 2007. The only reason I chose a single person for my top 10 is because I've been looking around for books to read for a while. You can find all my books on Amazon:
10 best fiction Kindle books
11 book review by Jodi Benson on her site:  Book review by  Jodi  Benson on  Kindle: A place for authors and readers
I had originally planned on doing an extensive list of all the books I read that were recommended to me by




,id,text,result
0,0,My Top 10 books of all time list :\n1. 1964\n2...,[ Is it true that My Top 10 books of all time ...


## SEO Text / Content Writing / Copy Writing
Lets generate some texts for our new imaginary shopify store!
We will sell some hoodies, soap and beard products.

Based on some base templates, we can generate high quality marketing/seo texts

In [ ]:
# from https://www.shopify.com/blog/8211159-9-simple-ways-to-write-product-descriptions-that-sell

# Lets define some base texts, from which we will generate new texts

hoodie = """Can’t stop buying plants? Unbeleafable. Don’t worry—us too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie! """

soap  = """Made with real pine extract, this all-star bar is as tough as a freshly cut bat.
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent."""

beard = """ Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control, """
gpt2.setTemperature(1)
gpt2.setDoSample(True)

GPT2TRANSFORMER_729a8e69e804

#### Marketing Texts for Hoodies

In [ ]:
# Lets generate some marketing texts for our hoodies
generate_with_gpt2(pipeline, [hoodie,hoodie,hoodie,hoodie,hoodie,hoodie] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Cant stop buying plants? Unbeleafable. Dont worryus too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie!  And if you do want any more information about this new line, here are some articles on its launch!
We're not sure if you've seen the Oodies, a very popular and incredibly expensive brand of products, but it's all about them: their Oodle and Oodles.  They are made using a proprietary process that uses plant-based ingredients and is the result of years of research and innovation. 
Their latest product, the I Love You Baby Oodlets, are made with a combination of plant proteins.  
It's an easy, fast and easy to use product that allows your body to use protein from its own tissue (not just from animal products) rather than rely on a va

,id,text,result
0,0,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
1,1,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
2,2,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
3,3,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
4,4,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
5,5,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...


#### Marketing Texts for Soap

In [ ]:
# Lets generate some marketing texts for our soap
generate_with_gpt2(pipeline, [soap,soap,soap,soap,soap,soap] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Made with real pine extract, this all-star bar is as tough as a freshly cut bat.
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent.


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Made with real pine extract, this all-star bar is as tough as a freshly cut bat.
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent.
If it weren't for this special blend of fresh, organic ingredients — including natural coconut extracts, coconut oil, and organic beeswax 

,id,text,result
0,0,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
1,1,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
2,2,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
3,3,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
4,4,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
5,5,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."


#### Marketing Texts for beard products

In [ ]:
# Lets generate some marketing texts for our beard products
generate_with_gpt2(pipeline, [beard,beard,beard,beard,beard,beard] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that  Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control,  while providing maximum protection against the elements so that it remains in place and looking and feeling natural. And to finish, with Beardbrand's patented formula , you're guaranteed to stay comfortable all day long, no matter how your hair grows.


Features:


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that  Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Design

,id,text,result
0,0,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
1,1,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
2,2,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
3,3,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
4,4,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
5,5,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."


## Song Lyrics

#### Last Christmas

In [ ]:
last_christmas = """
Last Christmas, I gave you my heart
But the very next day, you gave it away
This year, to save me from tears
I'll give it to someone special
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, last_christmas,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Last Christmas, I gave you my heart
But the very next day, you gave it away
This year, to save me from tears
I'll give it to someone special
I'd like to be honest with you, this isn't the most romantic poem that I've ever read. I don't care if it was written in the middle of a romantic movie or in an episode of The Voice.
But I do remember that my father had told me that he knew about this poem when he was visiting the country when my mother was pregnant with me.
He also said that this poem would make you laugh.
I remember my father, the kindhearted old man who would be able to make you feel like you were his only friend, saying that to me as I was crying in his arms, he thought, I'm sorry I couldn't keep it together the whole time.
My mom, whose family was al

,id,text,result
0,0,"\nLast Christmas, I gave you my heart\nBut the...","[ Is it true that \nLast Christmas, I gave you..."


#### Fresh prince of GPT

In [ ]:
bel_air = """
Now, this is a story all about how
My life got flipped-turned upside down
And I'd like to take a minute
Just sit right there
I'll tell you how I became the prince of a town called Bel-Air
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, bel_air,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Now, this is a story all about how
My life got flipped-turned upside down
And I'd like to take a minute
Just sit right there
I'll tell you how I became the prince of a town called Bel-Air
A town that used to have a king but no princes.
I mean, I used to live there
But then I found out about
The man that used
To be my hero, the one-man army
That's been going around killing the peasants
They said my parents got me
In a cage and tortured me
I guess my mother was afraid
That this hero would be too much
Then I fell in love with
The guy,
And after I saved one man
I came over to visit him.
The next day I'd find him in a
Fully clothed bathing suit
Looking like he was just
He looked a little bit bored and bored
After all the people who had come
To talk to him, he'd alr

,id,text,result
0,0,"\nNow, this is a story all about how\nMy life ...","[ Is it true that \nNow, this is a story all a..."


#### GPT Jackson

In [ ]:
thriller_lyrics = """
They told him, "Don't you ever come around here"
"Don't wanna see your face, you better disappear"
The fire's in their eyes and their words are really clear
So beat it, just beat it
You better run, you better do what you can
Don't wanna see no blood, don't be a macho man
You wanna be tough, better do what you can
So beat it, but you wanna be bad
Just beat it (beat it), beat it (beat it)
No one wants to be defeated
Showin' how funky and strong is your fight
It doesn't matter who's wrong or right
Just beat it (beat it)
Just beat it (beat it)
"""


gpt2.setMaxOutputLength(500)
generate_with_gpt2(pipeline, thriller_lyrics,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
They told him, "Don't you ever come around here"
"Don't wanna see your face, you better disappear"
The fire's in their eyes and their words are really clear
So beat it, just beat it
You better run, you better do what you can
Don't wanna see no blood, don't be a macho man
You wanna be tough, better do what you can
So beat it, but you wanna be bad
Just beat it (beat it), beat it (beat it)
No one wants to be defeated
Showin' how funky and strong is your fight
It doesn't matter who's wrong or right
Just beat it (beat it)
Just beat it (beat it)
So if you wanna lose, beat it!
(beat it, beat the shit, be the baddest)
That's why you need a little bit of help
And if you ain't got no friends on the outside
You have to fight hard
No friend is bigger than yourself
So I'm 

,id,text,result
0,0,"\nThey told him, ""Don't you ever come around h...","[ Is it true that \nThey told him, ""Don't you ..."


#### Real Slim GPT

In [ ]:
real_slim_shady = """
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here?
Y'all act like you never seen a white person before
Jaws all on the floor like Pam, like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (ah!)
It's the return of the, oh wait, no way, you're kidding
He didn't just say what I think he did, did he?
And Dr. Dre said-, nothing you idiots
Dr. Dre's dead, he's locked in my basement (haha)
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, real_slim_shady,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here?
Y'all act like you never seen a white person before
Jaws all on the floor like Pam, like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (ah!)
It's the return of the, oh wait, no way, you're kidding
He didn't just say what I think he did, did he?
And Dr. Dre said-, nothing you idiots
Dr. Dre's dead, he's locked in my basement (haha)
I mean, you all know that's a lie
We never got into a fight with that guy
We used to live next door and he kept us in our closet
But that's the way it was, 

,id,text,result
0,0,"\nMay I have your attention, please?\nMay I ha...","[ Is it true that \nMay I have your attention,..."


#### GPT Rap God Bot

In [ ]:
rap_god_lyrics = """
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance (six minutes-, six minutes-)
Something's wrong, I can feel it (six minutes, Slim Shady, you're on!)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means what I think it means, we're in trouble, big trouble
And if he is as bananas as you say, I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot """

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, rap_god_lyrics,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance (six minutes-, six minutes-)
Something's wrong, I can feel it (six minutes, Slim Shady, you're on!)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means what I think it means, we're in trouble, big trouble
And if he is as bananas as you say, I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot _____
I don't even rap like the first guy
I am so rap that you wouldn't believe the truth
I can go through a

,id,text,result
0,0,"\nLook, I was gonna go easy on you not to hurt...","[ Is it true that \nLook, I was gonna go easy ..."


## Fiction Writing

#### Fantasy Stories

In [ ]:
unicorn_story = """In a shocking finding, scientist discovered a herd of unicorns living in a remote,
previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English."""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, unicorn_story,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that In a shocking finding, scientist discovered a herd of unicorns living in a remote,
previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English. According to the study, the unicorn's dialect and vocabulary included many words found in American Spanish spoken in Mexico. "It is the first known instance of communication between a unicorn and a native English speaker," researchers explained. The findings were published in the scientific journal, International Journal of the Andean Biodiversity.
This is not the first time that a herd in Mexico has spoken perfect English with an American accent: According to researchers at the University of Texas – Austin, a herd from Argentina has 

,id,text,result
0,0,"In a shocking finding, scientist discovered a ...","[ Is it true that In a shocking finding, scien..."


In [ ]:
silly_story = """
Blue creatuers have been found in underground cavern in Maxico, which are mysteriously filled with peanut butter pockets.
These pockets are rich in proteins and enabled a civilization of these creatures to flurish and
build technologies which have surpassed human understanding. Scientist have studied these
creataures and found incredible discoveries, including their mastery of the Spanish language and perfect Tacos!"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, silly_story,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Blue creatuers have been found in underground cavern in Maxico, which are mysteriously filled with peanut butter pockets.
These pockets are rich in proteins and enabled a civilization of these creatures to flurish and
build technologies which have surpassed human understanding. Scientist have studied these
creataures and found incredible discoveries, including their mastery of the Spanish language and perfect Tacos!




,id,text,result
0,0,\nBlue creatuers have been found in undergroun...,[ Is it true that \nBlue creatuers have been f...


#### Lovecraftian Horror Stories

In [ ]:
lovecraft_text = """
At this horror I sank nearly to the lichened earth,
transfixed with a dread not of this nor any world, but only of the mad spaces between the stars.

Out of the unimaginable blackness beyond the gangrenous glare of that cold flame,
out of the Tartarean leagues through which that oily river rolled uncanny, unheard, and unsuspected,
there flopped rhythmically a horde of tame, trained, hybrid winged things that no sound eye could ever wholly grasp,
or sound brain ever wholly remember. They were not altogether crows, nor moles, nor buzzards, nor ants,
nor vampire bats, nor decomposed human beings; but something I cannot and must not recall."""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, lovecraft_text,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
At this horror I sank nearly to the lichened earth,
transfixed with a dread not of this nor any world, but only of the mad spaces between the stars.

Out of the unimaginable blackness beyond the gangrenous glare of that cold flame,
out of the Tartarean leagues through which that oily river rolled uncanny, unheard, and unsuspected,
there flopped rhythmically a horde of tame, trained, hybrid winged things that no sound eye could ever wholly grasp,
or sound brain ever wholly remember. They were not altogether crows, nor moles, nor buzzards, nor ants,
nor vampire bats, nor decomposed human beings; but something I cannot and must not recall.
, I could not and would not remember, because they were, and were not, so many different species of winged thing. It was the 

,id,text,result
0,0,\nAt this horror I sank nearly to the lichened...,[ Is it true that \nAt this horror I sank near...


In [ ]:
cthulu_intro = """
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents.
We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far.
The sciences, each straining in its own direction, have hitherto harmed us little"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, cthulu_intro,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents.
We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far.
The sciences, each straining in its own direction, have hitherto harmed us little.
For, though their methods are well calculated, yet their effects are of a nature to be feared.
It is very easy to give a scientific name to such an effect.
"But, O Lord, the effect, I pray thee, does not exist!
Why, why, why; why, the sun has never appeared in human eyes!
What do we owe to Him, if He does not appear?"
"What do you mean? No; it cannot be. You cannot. You must not say; what shall I do?
What shall I say? I ask you to stop your h

,id,text,result
0,0,"\nThe most merciful thing in the world, I thin...",[ Is it true that \nThe most merciful thing in...


#### Re-Write movie scripts

In [ ]:
the_matrix_script = """
The Architect: Hello, Neo.

Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: """

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, the_matrix_script,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
The Architect: Hello, Neo.

Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect:  The  Matrix .

Conconci-

Err, you see, the Matrix  is a massive, complex database and database simulation machine. You may have heard of it before, but it has many more functions:

It is the  Machine .  You can create or destroy it at will. It is a giant virtual computer of infinite scale, and

,id,text,result
0,0,"\nThe Architect: Hello, Neo.\n\nNeo: Who are y...","[ Is it true that \nThe Architect: Hello, Neo...."


In [ ]:
the_matrix_script = """
The Architect: Hello, Neo.
Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: """

gpt2.setMaxOutputLength(500)
generate_with_gpt2(pipeline, the_matrix_script,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
The Architect: Hello, Neo.
Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect:  I am not here to tell you. You are here to destroy me.
This is why we cannot communicate. This is why you must have a clear understanding of what is going on here, before you begin to act.
Now, let me ask you a few questions, that will clarify your current situation as far as my plans go.
First,

,id,text,result
0,0,"\nThe Architect: Hello, Neo.\nNeo: Who are you...","[ Is it true that \nThe Architect: Hello, Neo...."
